<a href="https://colab.research.google.com/github/Zhejing-Chin/fyp/blob/main/Extract_Objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mat4py


# Convert XML to CSV

In [ ]:
# Convert XML to CSV
import os
import xml.etree.ElementTree as ET
import csv

DATASET_DIR = '/content/drive/My Drive/FYP/New Data/2nd Batch [VillageGrocer]'
ANNOTATIONS_FILE = DATASET_DIR + '/NewDataAnnotations.csv'
CLASSES_FILE = DATASET_DIR + '/NewDataClasses.csv'
annotations = []
classes = set([])

for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  file_name = None

  for elem in root:
    if elem.tag == 'filename':
      file_name = elem.text # os.path.join(DATASET_DIR, elem.text)

    if elem.tag == 'size':
      for subelem in elem:
        if subelem.tag == 'width':
          width = subelem.text
        if subelem.tag == 'height':
          height = subelem.text

    if elem.tag == 'object':
      obj_name = None
      coords = []
      for subelem in elem:
        if subelem.tag == 'name':
          obj_name = subelem.text
        if subelem.tag == 'bndbox':
          for subsubelem in subelem:
            coords.append(subsubelem.text)
      item = [file_name] + coords + [obj_name] + [width] + [height]
      annotations.append(item)
      classes.add(obj_name)

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)
  print("annotations file created")

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))
  print("class file created")

annotations file created
class file created


# Extract Objects for training

In [ ]:
def showSubplotImage(img, save_path, titlestr=""):
    # plt.clf()
    print(titlestr)
    plt.imshow(img, interpolation='bilinear')
    plt.xticks([]), plt.yticks([])  
    try:
      os.makedirs(save_path)
    except OSError as e:
      if e.errno != errno.EEXIST:
          raise
    plt.savefig(save_path+'/'+titlestr,bbox_inches='tight',transparent=True, pad_inches=0)
    
    plt.title(titlestr)
    plt.close()
    # plt.show()

In [ ]:
# Extract objects
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import PIL
# from mat4py import loadmat
import re 
import errno, os
plt.ioff()

# class_index = loadmat('/content/drive/MyDrive/FYP/Grocery_products/Training/TrainingClassesIndex.mat')
save_path = '/content/drive/MyDrive/FYP/extracted objects/GP/'
image_path = '/content/drive/MyDrive/FYP/Grocery_products/Testing/store5/' #.format(str(j))

annotation = pd.read_csv(image_path + 'annotations/annotations_test.csv',
                        usecols = [i for i in range(8)],
                        # header=0)
                        names=["image_name", "x1", "y1", "x2", "y2", "class_", "width", "height"])

# print(annotation.dtypes)
for row in annotation.itertuples():
  # #read gp dataset classes--
  # annotation.at[row.Index, 'class'] = class_index['classes'][row.class_-1].split('/')[-1]
  # annotation = annotation[["image_path", "x1", "y1", "x2", "y2", "class", "width", "height"]]
  # annotation.to_csv(save_path + 'annotations_test.csv', index=False, header=False)

  # Extract products--
  if (row.class_ != 'Cheese'):
    continue
  # subpath = str(class_index['classes'][row.class_-1])+'/'
  # sub_path = "test/"+row.class_+'/'

  # image = PIL.Image.open(image_path + row.image_name)
  # w, h = image.size

  img = cv2.imread(image_path + 'images/' + row.image_name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  # showSubplotImage(img[abs(int(row.y1*h)):abs(int(row.y2*h)), 
  #                      abs(int(row.x1*w)):abs(int(row.x2*w))], sub_path, str(row.Index))
  try:
    showSubplotImage(img[row.y1:row.y2, row.x1:row.x2], save_path+'/'+row.class_, str(row.Index)+'_'+row.image_name)
  except ValueError:  
    pass


174_store5_88.jpg
175_store5_88.jpg
176_store5_88.jpg
177_store5_88.jpg
178_store5_88.jpg
179_store5_88.jpg
180_store5_158.jpg
181_store5_158.jpg
182_store5_158.jpg
183_store5_158.jpg
184_store5_158.jpg
202_store5_159.jpg
203_store5_159.jpg
204_store5_159.jpg
205_store5_159.jpg
206_store5_89.jpg
207_store5_89.jpg
208_store5_89.jpg
209_store5_89.jpg
210_store5_89.jpg
211_store5_89.jpg
333_store5_215.jpg
334_store5_215.jpg
335_store5_215.jpg
336_store5_215.jpg
345_store5_217.jpg
346_store5_217.jpg
347_store5_217.jpg
348_store5_149.jpg
349_store5_149.jpg
359_store5_161.jpg
360_store5_161.jpg
361_store5_161.jpg
379_store5_160.jpg
380_store5_160.jpg
381_store5_160.jpg
382_store5_160.jpg
383_store5_160.jpg
387_store5_98.jpg
388_store5_98.jpg
389_store5_98.jpg
390_store5_98.jpg
391_store5_98.jpg
392_store5_148.jpg
393_store5_216.jpg
394_store5_216.jpg
395_store5_216.jpg
406_store5_151.jpg
407_store5_151.jpg
408_store5_151.jpg
409_store5_95.jpg
410_store5_95.jpg
411_store5_95.jpg
412_store5_95

In [ ]:
# CSV file
detection_path = '/content/drive/MyDrive/FYP/detection_results_filtered_iou/'
csv = 'res_store_4_0.5_2021-01-23 05:16:01.089029.csv' 

save_path = detection_path + csv.split('.csv')[0]
create_folder(save_path)
annotation = pd.read_csv(detection_path + csv,
                         usecols = [i for i in range(6)],
                         header = None,
                         names=["image_name", "x1", "y1", "x2", "y2", "class_"])

for row in annotation.itertuples():
  subpath = row.image_name.split("_")[0]
  image_path = '/content/drive/MyDrive/FYP/Grocery_products/Testing/{}/images/'.format(subpath)
  img = cv2.imread(image_path + row.image_name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  try:
    showSubplotImage(img[row.y1:row.y2, row.x1:row.x2], save_path+'/'+row.class_, str(row.Index)+'_'+row.image_name)
  except ValueError:  
    pass
  # break

0_store4_14.jpg
1_store4_14.jpg
2_store4_14.jpg
3_store4_29.jpg
4_store4_15.jpg
5_store4_15.jpg
6_store4_15.jpg
7_store4_15.jpg
8_store4_17.jpg
9_store4_17.jpg
10_store4_16.jpg
11_store4_16.jpg
12_store4_16.jpg
13_store4_12.jpg
14_store4_12.jpg
15_store4_12.jpg
16_store4_11.jpg
17_store4_11.jpg
18_store4_11.jpg
19_store4_11.jpg
20_store4_10.jpg
21_store4_10.jpg
22_store4_77.jpg
23_store4_77.jpg
24_store4_77.jpg
25_store4_63.jpg
26_store4_63.jpg
27_store4_62.jpg
28_store4_62.jpg
29_store4_76.jpg
30_store4_60.jpg
31_store4_60.jpg
32_store4_74.jpg
33_store4_48.jpg
34_store4_48.jpg
35_store4_75.jpg
36_store4_75.jpg
37_store4_61.jpg
38_store4_59.jpg
39_store4_59.jpg
40_store4_65.jpg
41_store4_65.jpg
42_store4_65.jpg
43_store4_71.jpg
44_store4_71.jpg
45_store4_70.jpg
46_store4_64.jpg
47_store4_58.jpg
48_store4_8.jpg
49_store4_8.jpg
50_store4_72.jpg
51_store4_72.jpg
52_store4_72.jpg
53_store4_72.jpg
54_store4_72.jpg
55_store4_72.jpg
56_store4_66.jpg
57_store4_66.jpg
58_store4_67.jpg
59_store4